In [1]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib

# ランダムな交換により現れる分布について

ランダムな交換や分配によって現れる分布について、プログラムによるシミュレーションを行った。

シミュレーションの中にはかなり時間がかかるものもある。設定されているループの繰り返しの回数は、表示されている結果を得るために必要な回数である。

分布がどのように変わっていくのかを見たい場合は、頭にある変数の初期化部分からループ以降の部分を切り離し、繰り返しの回数を少なくして少しずつやっていくことをお勧めする。

きっかけとなったのは板尾健司氏による以下の記事。

[ランダムな世界では「所得の平等」は実現しない…数理モデルが解き明かした、超富裕層が生まれる仕組み](https://gendai.media/articles/-/148324)


## 1. ランダムな交換（マイナスを許す）

みな同じだけの財産を持っている状態から始めて、ランダムに選んだ二人が一定量の財を交換する（一方が他方に贈与する）。
与える側に多く選ばれた場合、財産はマイナスになることもある。

結果は正規分布になる。

<img src='image/dist1.png'>

In [15]:
def normal(x, u, v):
    ret = 1 / np.sqrt(2 * np.pi * v) * np.exp(-(x-u)**2/(2*v))
    return ret

In [ ]:
N = 10000 # 人の数
w0 = 100 # 初期の財産

w = np.full(N, w0, dtype=np.int32)
num_iter = 0

for i in range(50000000):
    # ランダムに二人を選ぶ
    x = random.sample(range(N), k=2)
    w[x[0]] -= 1 # 一方に 1 を足す
    w[x[1]] += 1 # 他方から 1 を引く
    num_iter += 1

plt.hist(w, bins=100)
x = np.arange(-200, 401, 10)
plt.plot(x, 74000 * normal(x, 100, 10000), label='正規分布')
plt.xlim(-200, 400)
plt.xlabel("財産")
plt.ylabel("人数")
plt.legend()
plt.show()
print(f'繰り返し数: {num_iter}')

### 考察
一人の人に注目すると、これは結局「同確率で +1 か -1 する」という試行を繰り返しているにすぎない。よってその分布は二項分布となり、試行の回数を増やせば正規分布に近づいていく。

試行を繰り返すと、分布の標準偏差は、試行回数を $M$ として $\sqrt{M}$ に比例して増加する。従って分布の幅もそれに合わせて徐々に広がってゆく。


## 2. ランダムな交換（多い方に１を足し、少ない方から１を引く）

ランダムに二人を選ぶことは同じで、現在の財産が少ないほうが多い方に贈与する。

結果は一様分布になる。

<img src='image/dist8.png'>

In [ ]:
N = 10000 # 人の数
w0 = 100 # 初期の財産

w = np.full(N, w0, dtype=np.int32)
num_iter = 0

for i in range(10000000):
    # ランダムに x, y を選ぶ。
    x, y = tuple(random.sample(range(N), k=2))
    # w[x] > w[y]　なら x と y を入れ替える
    if w[x] > w[y]:
        x, y = y, x
    w[x] -= 1 # 少ない方から１を引く
    w[y] += 1 # 多い方に１を足す
    num_iter += 1

plt.hist(w, bins=100)
plt.xlabel("財産")
plt.ylabel("人数")
plt.show()
print(f'繰り返し数: {num_iter}')

### 考察

この結果は一見したところ直感に反するかもしれない。分布の台形の両端はますます左右に離れてゆき、中央あたりは財産の増減が打ち消し合うから、中央が盛り上がった曲線になるのではないかと予想するかもしれない。だが、次のように考えれば分布が水平な台形型になることが納得できるかと思う。

まず、ある時点で分布が台地型になっていると仮定する。

一回の交換で、現在ある量の財産を持つ人が財を得る確率は、台地の左端だとゼロ、右端だと 1 で、その間は距離に**比例して**変わっていく。

財を減らす確率は、増える場合の左右を逆にしたものとなる。

つまり、現在ある量の財を持つ人が、その財を変化させる確率=(1増えて右へ移動する確率)+(1減って左へ移動する確率)となり、これは財産によらず一定である。

一方、ある量の財を持つ人の数がよそから加わって増える確率=(財産が1多い人が1減らす確率)+(財産が1少ない人が1増やす確率)となり、これも財産にはよらない一定数となる。

つまり、ある財産を持つ人の増減の期待値は財産の量によらず一定となる。従って分布の台地形は保たれたままとなる。ただ、台地の高さは一定の高さを保ちつつ低くなってゆき、分布の幅は左右に広がっていく。

このように、交換において貧富の差を拡大するような力が働く時、分布は外へと無限に広がっていく。この場合、それ以上は変化せず安定する、という状態に達することはない。

試行によって分布の平均 $C$ は変わらないが、この平均との差の絶対値を全員について足し合わせたものを $D$ とすると、ある試行において、

- x と y が両方とも $C$ 以下
- x と y が両方とも $C$ より大きい
- x と y の一方が $C$ 以下で、他方は $C$ より大きい

という事象は 1:1:2 の割合で起こり得るが、最初と２番目では $D$ は変化せず、三番目では 2 増える。
つまり $D$ の期待値は試行ごとに 1 だけ増えていく。
よって、ばらつきの大きさは試行回数 $M$ に比例して増加していく。


## 3. ランダムな交換（ゼロの人からは取らない）

ランダムに選んだ二人が一定量の財を交換するが、ただし財産がゼロの人からは取らない（交換を中止し、選択をやり直す）。
このため財産はゼロより少なくなることはない。

結果は指数分布になる。

<img src='image/dist2.png'>

こちらは y 軸を対数目盛にしたもの。

<img src='image/dist2_2.png'>

In [ ]:
N = 10000 # 人の数
w0 = 100 # 初期の財産

w = np.full(N, w0, dtype=np.int32)
num_iter = 0

for i in range(200000000):
    # ランダムに x, y を選ぶ。
    # ただし x の財産がゼロならやり直す。
    x, y = tuple(random.sample(range(N), k=2))
    while w[x] == 0:
        x, y = tuple(random.sample(range(N), k=2))
    w[x] -= 1
    w[y] += 1
    num_iter += 1

print(f'繰り返し数: {num_iter}')

plt.hist(w, bins=665)
x = range(0, 601, 10)
plt.plot(x, 95 * np.power(0.99, x), label='y=0.99^x')
plt.xlim(0, 600)
plt.ylim(0, 150)
plt.xlabel("財産")
plt.ylabel("人数")
plt.legend()
plt.show()

plt.hist(w, bins=665)
plt.yscale("log")
x = range(0, 601, 10)
plt.plot(x, 95 * np.power(0.99, x), label='y=0.99^x')
plt.xlim(0, 600)
plt.xlabel("財産")
plt.ylabel("人数")
plt.legend()
plt.show()

### 考察

この結果については、厳密ではないが、以下のように考えてみることができる。

人の数を $N$、財産が $x$ の人の人数を $ f(x) $ とすると、一回の試行による $ f(x) $ の増分は、

$x >= 1$ については

$$
\frac{1}{N}f(x-1) + \frac{1}{N-f(0)}f(x+1) - \frac{1}{N}f(x) - \frac{1}{N-f(0)}f(x)
$$
$$
= (\frac{1}{N-f(0)}f(x+1) - \frac{1}{N}f(x)) - (\frac{1}{N-f(0)}f(x) - \frac{1}{N}f(x-1))
$$

$x = 0$ については

$$ \frac{1}{N-f(0)}f(1) - \frac{1}{N}f(0) $$

となる。

ここで、試しに $ f(x) = f(0) \cdot c^x $ と置くと、これらの増分は

$$ \frac{c}{N-f(0)} - \frac{1}{N} = 0 $$

であればゼロになることが分かる。つまり、

$$ c = \frac{N-f(0)}{N} $$

であればよい。

上の図では、この関係をほぼ満たしている。

もちろんこの説明は $ f(x) = f(0) \cdot c^x $ であればうまくいくと言っているにすぎず、また、全員同じ財産から始めてそのような分布に近づいていくことも証明していない。

この交換は「1. ランダムな交換（マイナスを許す）」に「財産ゼロのひとからは取らない」という条件を加えただけなのだが、結果の分布は正規分布から指数分布へと変化する。加えられた条件が分布の広がりを抑える方向に働くのだ。

実際、この交換では財産の最小値はゼロ、最大値も実質的にはほぼ 700 以下に抑えられる。（「実質的」の意味は、原理的な上限はないが、指数関数的なバリアによって 700 を超えるのは稀であるということ。）


## 4. ランダムに人を選び、資産を×÷ (1 + r) する

結果は対数正規分布になる。

<img src='image/dist7.png'>

In [803]:
N = 10000 # 人の数
r = 0.005
# 初期の財産は 1 とする。

w = np.ones(N)
num_iter = 0

for i in range(50000000):
    # ランダムに人を選ぶ
    x = random.randrange(N)
    # コイン投げ
    zero_one = random.randrange(2)
    if zero_one == 0:
        # その人の資産を 1 + r 倍する
        w[x] *= (1 + r)
    else:
        # その人の資産を 1 + r で割る
        w[x] /= (1 + r)
    num_iter += 1

plt.hist(w, bins=100)
plt.xlabel("財産")
plt.ylabel("人数")
plt.show()
print(f'繰り返し数: {num_iter}')

### 考察

掛けるか割るかを決めるためのコイン投げにおいて、表を +1、裏を -1 として合計を取ると、それは二項分布≒正規分布となる。
つまり、初期値に $1+r$ を掛ける回数（割る場合はマイナスと考える）は正規分布に従う。
値の対数を取ると、それは掛ける回数に比例する数となり、正規分布となる。よって元の値は（「対数正規分布」の定義により）対数正規分布に従う。

対数正規分布の分散は $ e^{2\mu+\sigma^2}(e^{\sigma^2} - 1) $ であり、$\sigma^2$ が試行回数 $M$ に比例して増加する時、この分散はほぼ $(e^{M})^2$ に比例して増加する。従って標準偏差は $e^{M}$ に比例して増大する。つまり、正規分布の場合とは異なり、分布のばらつきは急速に増加していく。


## 5. ランダムに人を選び、資産を 1 + r 倍する

これは「4. ランダムに人を選び、資産を×÷ (1 + r) する」の変形バージョンで、表を +1、裏を 0（なにもしない）と考えたときに相当する。
結果はやはり対数正規分布になる。

以下のグラフは x が対数目盛り。

<img src='image/dist4.png'>

In [16]:
def log_normal_distribution(x, mu, sigma_2):
    r = np.log(x) - mu
    return ((1.0 / math.sqrt(2.0 * math.pi * sigma_2)) * x) * np.exp(-0.5 / sigma_2 * (r * r))

In [ ]:
N = 10000 # 人の数
r = 0.005
# 初期の財産は 1 とする。

w = np.ones(N)
num_iter = 0

for i in range(1):#10000000
    # ランダムに人を選ぶ
    x = random.randrange(N)
    # その人の資産を 1 + r 倍する
    w[x] *= (1 + r)
    num_iter += 1

plt.hist(w, bins=np.logspace(1.95, 2.4, 209))
plt.xscale("log")
x = np.arange(80, 301, 5)
plt.plot(x, 0.34 * log_normal_distribution(x, 4.96, 0.023), label='対数正規分布')
plt.xlabel("財産")
plt.ylabel("人数")
plt.legend()
plt.show()
print(f'繰り返し数: {num_iter}')

## 6. 資産に比例した確率でランダムに 1 与える

結果は対数正規分布になる。

<img src='image/dist3.png'>

In [ ]:
N = 10000 # 人の数
w0 = 10 # 初期の財産

w = np.full(N, w0, dtype=np.int32)
num_iter = 0

range_N = range(N)
for i in range(700000):
    # 資産に比例した確率で受け取る人を選ぶ
    x = random.choices(range_N, weights=w)[0]
    # 1 与える。
    w[x] += 1
    num_iter += 1
    
plt.hist(w, bins=199)
x = np.arange(10, 201, 2)
plt.plot(x, 1.35 * log_normal_distribution(x+10, 4.36, 0.06), label='対数正規分布')
plt.xlabel("財産")
plt.ylabel("人数")
plt.legend()
plt.show()
print(f'繰り返し数: {num_iter}')

### 考察

理由は不明。分かる方がおられたらご一報いただけると幸いです。

ただランダムに選んだ人に 1 を与えるだけなら正規分布になるはずだが、資産に比例した確率で選ぶと対数正規分布になるらしい。


## 7. 資産に比例した確率で人を選び、資産を 1 + r 倍する

結果はやはり対数正規分布になる。

<img src='image/dist5.png'>

In [76]:
N = 10000 # 人の数
r = 0.02
# 初期の財産は 1 とする。

w = np.ones(N)
num_iter = 0
range_N = range(N)

# 資産に比例した確率で人を選び、資産を 1 + r 倍する
for i in range(200000):
    # 資産に比例した確率で人を選ぶ
    x = random.choices(range_N, weights=w)[0]
    w[x] *= (1 + r)
    num_iter += 1

plt.hist(w, bins=51)
plt.xlabel("財産")
plt.ylabel("人数")
plt.show()
print(f'繰り返し数: {num_iter}')

## 8. ランダムな取り引き。富が少ない方は 1 + r で割り、多い方は掛ける

結果はべき分布（$\frac{1}{x}$ に比例）になる。

<img src='image/dist6.png'>

以下のグラフは x が対数目盛り。

<img src='image/dist6_2.png'>

In [ ]:
N = 10000 # 人の数
r = 0.001
# 初期の財産は 1 とする。

w = np.ones(N)
num_iter = 0

for i in range(20000000):
    # ランダムに x, y を選ぶ。
    x, y = tuple(random.sample(range(N), k=2))
    # x の財産が少なくなるよう、必要なら入れ替える
    if w[x] > w[y]:
        x, y = y, x
    w[x] /= (1 + r) # x の財産を 1+r で割る
    w[y] *= (1 + r) # y の財産に 1+r 掛ける
    num_iter += 1

plt.hist(w, bins=500)
x = np.arange(1, 71, 1)
plt.plot(x, 140 / x, label='y=140/x')
plt.xlim(0, 70)
plt.ylim(0, 100)
plt.xlabel("財産")
plt.ylabel("人数")
plt.legend()
plt.show()
    
plt.hist(w, bins=np.logspace(-2, 2, 500))
plt.xscale("log")
plt.xlim(0.01, 100)
plt.xlabel("財産")
plt.ylabel("人数")
plt.show()
print(f'繰り返し数: {num_iter}')

### 考察

$ 1 + r $ を掛けたり割ったりするというのは、$ \log $ を取れば $ \log(1+r) $ を足し引きすることに等しい。従ってその結果は「2. ランダムな交換（多い方に１を足し、少ない方から１を引く）」で述べた一様分布になり、元の値に戻せばそれは$\frac{1}{x}$ に比例する「べき分布」となる（下のコラム参照）。

この結果は、「2. ランダムな交換（多い方に１を足し、少ない方から１を引く）」を「加法バージョン」と考えたとき，その乗法バージョンと言える。どちらも取り引きにおいて貧富の差を大きくするような力が働いている。財産の分布はどこかで安定することはなく、貧富の差は際限なく拡大していく。

また、分布のばらつきは log を取った値で考えれば試行回数 $M$ に比例して増加するので、元の値では $e^M$ に比例して増えていくと考えられる。


### コラム: 対数を取った値による分布と元の分布の関係

一般に、元の値の log を x 軸に取って描いた分布関数（確率密度曲線）を $F(x)$ とするとき、元の値によって描かれた分布関数 $f(x)$ はどのようなものになるだろうか。

元の値が $x$ から $x+\varDelta x$ の間の値を取る確率は $\varDelta x$ が小さいとき $f(x) \cdot \varDelta x$ と近似できる。
これが、log を取った方のグラフでの確率 $F(\log(x)) \cdot (\log(x+\varDelta x) - \log(x))$ と等しくなければならないので、

$$
f(x) \cdot \varDelta x = F(log(x)) \cdot  (\log(x+\varDelta x) - \log(x))
$$

$\varDelta x$ を移項して

$$
f(x) = F(log(x)) \cdot  \frac{\log(x+\varDelta x) - \log(x)}{\varDelta x} 
$$

となる。ここで $\varDelta x$ を無限に小さくすると右辺の分数の部分は $\log(x)$ の微分、つまり $1/x$ となるので

$$
f(x) = \frac{F(log(x))}{x}
$$

となる。これが $F(x)$ から $f(x)$ を得る公式である。

例えば、上記「8. ランダムな取り引き。富が少ない方は 1 + r で割り、多い方は掛ける」の例では $F(x)$ は定数であり、従って $f(x)$ は反比例に比例する関数となる。

$F(x)$ が正規分布、つまり

$$
F(x) = {\frac {1}{\sqrt {2\pi \sigma ^{2}}}}\;\exp \left(-{\frac {\left(x-\mu \right)^{2}}{2\sigma ^{2}}}\right)
$$

の場合は、上記の変形によって $f(x)$ は

$$
f(x)={\frac {1}{{\sqrt {2\pi \sigma ^{2}}x}}}\exp \left(-{\frac {(\log {x}-\mu )^{2}}{2\sigma ^{2}}}\right)
$$

となり、対数正規分布になることが分かる。

$F(x)$ が指数関数の場合は、

$$
F(x) = k \cdot c^x
$$

を上記の式に当てはめると、

$$
\begin{align}
  f(x) &= \frac{k \cdot c^{\log(x)}}{x} \\
       &= \frac{k \cdot (e^{\log(c)})^{\log(x)}}{x} \\
       &= \frac{k \cdot e^{\log(c) \cdot \log(x)}}{x} \\
       &= \frac{k \cdot x^{\log(c)}}{x} \\
       &= k \cdot x^{\log(c) - 1}
\end{align}
$$

となり、べき分布になることが分かる。


## 9. ランダムな取り引き。一方を 1 + r 倍し、他方を 1 + r で割る（1 より減らさない）

ランダムに二人を選び、一方の財産を 1 + r 倍し、他方を 1 + r で割る。ただし財産が 1 以下の人は減らさない（交換を中止し、選択をやり直す）。 このため財産が 1 より少なくなることはない。

これは「3. ランダムな交換（ゼロの人からは取らない）」の乗法バージョンである。

結果はべき分布になる。

<img src='image/dist9.png'>

以下のグラフは x が対数目盛り。分布の形は指数分布になる。

<img src='image/dist9_2.png'>

以下のグラフは x, y の両方が対数目盛り。エッジは直線になる。

<img src='image/dist9_3.png'>

In [75]:
N = 10000 # 人の数
r = 0.05
w0 = 100 # 初期の財産

w = np.full(N, w0, dtype=float)
num_iter = 0

for i in range(200000000):
    # ランダムに x, y を選ぶ。
    # ただし x の財産が 1 以下ならやり直す。
    x, y = tuple(random.sample(range(N), k=2))
    while w[x] <= 1.0:
        x, y = tuple(random.sample(range(N), k=2))
    w[x] /= (1 + r) # x の財産を 1+r で割る
    w[y] *= (1 + r) # y の財産に 1+r 掛ける
    num_iter += 1

plt.hist(w, bins=np.arange(0, 201, 2))
x = np.arange(0.5, 201, 1)
plt.plot(x, 3000 * np.power(x, -1.1), label='y=k / x^1.1')
plt.xlim(0, 200)
plt.ylim(0, 1500)
plt.xlabel("財産")
plt.ylabel("人数")
plt.legend()
plt.show()

plt.hist(w, bins=np.logspace(0, 9, 214))
plt.xscale("log")
plt.xlim(1, 1000000000)
plt.xlabel("財産")
plt.ylabel("人数")
plt.show()

plt.hist(w, bins=np.logspace(0, 9, 200))
plt.xscale("log")
plt.yscale("log")
plt.xlim(1, 1000000000)
plt.xlabel("財産")
plt.ylabel("人数")
plt.show()

print(f'繰り返し数: {num_iter}')

### 考察

財産の log を取ると、これは「(1 + r) 倍された回数 - (1 + r) で割った回数」に比例する。つまり、その分布は「3. ランダムな交換（ゼロの人からは取らない）」で見たように指数分布となる。

log を取った値で表示された分布関数が指数関数となるとき、元の値での分布関数はべき関数となる（「コラム: 対数を取った値による分布と元の分布の関係」を参照）。

べき関数を x, y の両方を対数目盛にして表示すると、それは一次関数（直線）になる。

## まとめ

以上の結果をまとめると次のようになる。

1. ランダムな交換（マイナスを許す）
2. ランダムな交換（多い方に１を足し、少ない方から１を引く）
3. ランダムな交換（ゼロの人からは取らない）
4. ランダムに人を選び、資産を×÷ (1 + r) する
5. ランダムに人を選び、資産を 1 + r 倍する
6. 資産に比例した確率でランダムに 1 与える
7. 資産に比例した確率で人を選び、資産を 1 + r 倍する
8. ランダムな取り引き。富が少ない方は 1 + r で割り、多い方は掛ける
9. ランダムな取り引き。一方を 1 + r 倍し、他方を 1 + r で割る（1 より減らさない）

| 番号 | 加法的／乗法的 | 力 | 分布 | 広がり |
| ---- | ---- | ---- | ---- | ---- |
| 1 | 加法的 | 中立 | 正規分布 | $\sqrt{M}$ |
| 2 | 加法的 | 拡散力 | 一様分布 | $M$ |
| 3 | 加法的 | 収縮力 | 指数分布 | 実質的上限あり |
| 4,5,6,7 | 乗法的 | 中立 | 対数正規分布 | 急速に拡散（$e^M$） |
| 8 | 乗法的 | 拡散力 | べき分布（$\frac{1}{x}$） | 急速に拡散（$e^M$） |
| 9 | 乗法的 | 収縮力 | べき分布 | 急速に拡散（$e^M$）? |